In [50]:
import sys
sys.path.append("..")
from spark_wan.models.transformer_wan import WanTransformer3DModel
from spark_wan.training_utils.gan_utils import calculate_adaptive_weight
from peft import get_peft_model, LoraConfig
import torch

In [37]:
weight_dtype = torch.float16
transformer = WanTransformer3DModel.from_pretrained(
    "/storage/ysh/Ckpts/Wan2.1-T2V-1.3B-Diffusers/", subfolder="transformer", torch_dtype=weight_dtype
)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["ffn.net.2"],
    lora_dropout=0.1,
    inference_mode=False,
)
transformer.requires_grad_(False)
transformer = get_peft_model(transformer, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
transformer.enable_gradient_checkpointing()

In [39]:
transformer = transformer.to("cuda")

In [49]:
transformer.disable_adapter_layers()
teacher_output = transformer.forward(
    hidden_states=torch.randn(1, 16, 21, 80, 80, device="cuda", dtype=weight_dtype),
    timestep=torch.tensor([1000], device="cuda", dtype=torch.long),
    encoder_hidden_states=torch.randn(1, 512, 4096, device="cuda", dtype=weight_dtype),
    return_dict=False,
)[0]
transformer.enable_adapter_layers()

student_output = transformer.forward(
	hidden_states=torch.randn(1, 16, 21, 80, 80, device="cuda", dtype=weight_dtype),
	timestep=torch.tensor([1000], device="cuda", dtype=torch.long),
	encoder_hidden_states=torch.randn(1, 512, 4096, device="cuda", dtype=weight_dtype),
	return_dict=False
)[0]


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

: 